# Lab 2B: Functional programming, and declarative patterns


__Student:__ Sridhar Adhikarla(sriad858)

__Student:__ Anubhav Dikshit(anudi287)

Disclaimer: Functional programming in Python does not always lead to the fastest possible code, and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model which many big data systems, e.g. Hadoop, relies on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigm language, and functional programming is one of the main paradigms one can use. To understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements unless this is explicitly allowed in the task.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together). Incidentally, one might find speedups when using the immutable tuples instead of lists.

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

# 2 Recursion

As an introduction to linear recursion, read the introductory note on the course webpage. This might help explain terms that you may not know (even if the concept is previously known).

## 2.1 Linear recursion

a) Write a recursive function `sum_even(n)` that takes a natural number $n\geq 0$ and returns the sum of all even numbers $0,...,n$. It should be linear-recursive with delayed computations.

In [2]:
def sum_even(n):
    if n <= 1:
        return n
    else:
        if n%2==0:
            return n + sum_even(n-2)
        else:
            return sum_even(n-1)
    
print("Sum for n = 11 :", sum_even(11))
print("Sum for n = 100 :", sum_even(100))

Sum for n = 11 : 30
Sum for n = 100 : 2550


b) Write `sum_even_it(n)` according to the same specification. In this case, the solution should be tail recursive.

In [3]:
def sum_even_it(n, tot=0):
    if n <= 1:
        return tot
    else:
        if n%2==0:
            return sum_even_it(n-2, tot + n)
        else:
            return sum_even_it(n-1, tot)
        
print("Sum for n = 11 :", sum_even_it(11))
print("Sum for n = 100 :", sum_even_it(100))

Sum for n = 11 : 30
Sum for n = 100 : 2550


c) We can of course express this in a declarative and Pythonic way, which is non-recursive. Write a function `sum_even_py` which returns the same result as above, but using comprehension or filter/map/reduce construct.

In [4]:
from functools import reduce

def sum_even_py(n):
    return reduce((lambda x, y: (x%2==0)*x + (y%2==0)*y), range(n+1))

print("Sum for n = 11 :", sum_even_py(11))
print("Sum for n = 100 :", sum_even_py(100))

Sum for n = 11 : 30
Sum for n = 100 : 2550


## 2.2 Double/tree recursion

Sometimes we might find ourselves with branching structures, where there are several "smaller" cases to recurse over. This might for instance be the case when we have trees, lists-within-lists or the like.

[Note: In the tasks below, it might be helpful to gain an understanding of `isinstance`. See the documentation!]

a) One common use of recursion is to traverse recursive data structures. One exercise might be to _flatten_ nested lists or tuples. This is relatively simple with only one level of nesting, or when the structure follows a strict pattern, but for arbitrary nested sequences, a recursive approach is more natural. Implement a recursive function `myflatten` which can take an arbitrary structure of nested tuples and flattens it (in the sense of returning a new non-nested tuple with the same elements in the same order).

In [21]:
def myflatten(seq):
    if seq==():
        return ()
    elif not isinstance(seq, (list, tuple)):
        return (seq,)
    else:
        return myflatten(seq[0]) + myflatten(seq[1:]) 

def test_myflatten():
    tests = (
     ((), (), "the empty tuple"), 
     ((1,2,3), (1,2,3), "flat tuples invariant under flattening"), 
     ( (1, (2), 3, (4, 5, (6), 7), 8), (1, 2, 3, 4, 5, 6, 7, 8), "Arbitrarily nested tuples.")
    )
    for arg, expected_output, error_msg in tests:
        assert myflatten(arg) == expected_output, error_msg
    print("All Tests passed")

test_myflatten()    # Uncomment this line to run tests.

All Tests passed


b) Implement a function `exists_in(e, seq)` which returns `True` if the element `e` exists somewhere in the tuple `seq` (and `False` otherwise). `seq` might be nested and contain tuples-within-tuples.

In [22]:
def exists_in(e, seq):
    if seq==():
        return False
    elif not isinstance(seq, (list, tuple)):
        return seq==e
    else:
        return exists_in(e, seq[0]) or exists_in(e, seq[1:]) 

c) Write a few representative test cases, as in lab 2A.

In [23]:
def test_find_anywhere():
    tests = (
     (((), 10), False, "the empty tuple, nothing to find Should return False"), 
     (((1,2,3), 3), True, "Element 3 is thre in the tuple, should return True"), 
     ( ((1, (2), 3, (4, 5, (6), 7), 8), 5), True, "Arbitrarily nested tuples.Element 5 is thre in the tuple, should return True")
    )
    for arg, expected_output, error_msg in tests:
        assert exists_in(arg[1], arg[0]) == expected_output, error_msg
    print("All Tests passed")
    
test_find_anywhere()

All Tests passed


d) One of the most famous recursive functions is the Quicksort function (https://en.wikipedia.org/wiki/Quicksort). It allows us to sort a sequence, with repeated values, in (amortized) log-linear time and with a logarithmic number of recursive calls. We will start by implementing Quicksort for a tuple of numbers.

Note that Wikipedia illustrates a more advanced _in-place_ version of Quicksort, with a more advanced partition function. For the purposes of this assignment, you can simply pass a new tuple or generator to each recursive call to quicksort. You may use eg _filter_ or a comprehension to create the inputs.

In [8]:
from random import sample, choice

# Write quicksort here:
def partition(A, piv):
    A = list(A)
    temp = A[0]
    A[0] = A[piv]
    A[piv] = temp
    i = 1
    for j in range(1,len(A)):
        if A[j] < A[0]:
            if i==j:
                i += 1
            else:
                temp = A[j]
                A[j] = A[i]
                A[i] = temp
                i += 1
    temp = A[0]
    A[0] = A[i-1]
    A[i-1] = temp
    return A, i-1

def quicksort(A):
    n = len(A)
    if n<=1:
        return A
    else:
        piv = choice(range(n))
        A, ind = partition(A, piv)
        A[0:ind] = quicksort(A[0:ind])
        A[ind+1:] = quicksort(A[ind+1:])
        return tuple(A)
        
a = tuple(sample(range(1000,2000), 1000))
print(a)
b = quicksort(a)
print(b)

(1740, 1904, 1301, 1599, 1381, 1779, 1158, 1163, 1934, 1750, 1237, 1096, 1907, 1297, 1993, 1826, 1276, 1672, 1279, 1037, 1809, 1770, 1488, 1000, 1932, 1593, 1061, 1077, 1065, 1738, 1413, 1495, 1571, 1307, 1578, 1159, 1283, 1359, 1051, 1802, 1763, 1539, 1373, 1013, 1304, 1564, 1050, 1592, 1407, 1365, 1136, 1195, 1811, 1523, 1244, 1541, 1438, 1006, 1853, 1215, 1041, 1436, 1394, 1748, 1767, 1018, 1688, 1046, 1331, 1228, 1985, 1223, 1116, 1687, 1153, 1518, 1801, 1522, 1151, 1547, 1099, 1125, 1743, 1508, 1856, 1305, 1404, 1603, 1874, 1914, 1154, 1479, 1231, 1505, 1224, 1957, 1363, 1088, 1176, 1782, 1772, 1967, 1207, 1059, 1864, 1482, 1894, 1700, 1167, 1552, 1569, 1974, 1621, 1504, 1155, 1735, 1731, 1086, 1282, 1931, 1327, 1193, 1833, 1075, 1653, 1001, 1494, 1813, 1227, 1316, 1938, 1984, 1132, 1156, 1929, 1555, 1560, 1091, 1662, 1695, 1943, 1964, 1668, 1202, 1575, 1321, 1027, 1364, 1355, 1902, 1785, 1845, 1713, 1947, 1102, 1481, 1318, 1390, 1314, 1422, 1287, 1108, 1045, 1780, 1005, 1918, 184

## 3 Higher order functions (HOF)
 
A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, but we will call any function which returns a function or takes a function as an argument a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_)

In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's [$\lambda$-calculus](https://www.youtube.com/watch?v=eis11j_iGMs) which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

## 3.1 The three standard functions `map`, `reduce` and `filter`

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard built-in functions. Since Python 3, the `reduce` function needs to be imported from the `functools` module.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

A note from last year: usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by André Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

a) Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [8]:
from functools import reduce

def mysum(seq):
    return reduce((lambda x, y: x + y), seq)

mysum((4, 7, 1))

12

b) Implement a function `mylength` which uses `map` and `reduce` to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: Use `map` to convert the input to something which can easily be `reduce`:d.]

In [9]:
def mylength(seq):
    return reduce((lambda x, y: x + y), map(lambda x: 1, seq))
    
print(mylength((4, 2, 5, 2, 5)))
print(mylength("test"))

5
4


## 3.2 Building your own higher order functions

a) Re-implement the three basic functional helper functions `map`, `filter` and `reduce` **as purely functional recursive functions**. You may not express this as eg comprehensions; the task is to practice figuring out this type of logic.

Note that the built-in versions of these functions work on multiple sequences of equal length if supplied, however, you can assume a single sequence as second parameter, i.e. you can also skip the third parameter to reduce.

In [24]:
def mymap(f, seq):
    if seq==[]:
        return []
    elif not isinstance(seq, (list, tuple)):
        return [f(seq),]
    else:
        return mymap(f, seq[0]) + mymap(f, seq[1:])

mymap(lambda x:x**2, list(range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [25]:
def myfilter(f, seq):
    if seq==[]:
        return []
    elif not isinstance(seq, (list, tuple)):
        if f(seq):
            return [seq,]
        else:
            return []
    else:
        return myfilter(f, seq[0]) + myfilter(f, seq[1:])

myfilter(lambda x:x%2==0, list(range(10)))

[0, 2, 4, 6, 8]

You might note the similarities with how you implemented `sum_even`.

In [26]:
def myreduce(f, seq):
    if seq==[]:
        return []
    elif not isinstance(seq, (list, tuple)):
        return seq
    else:
        first = myreduce(f, seq[0])
        last = myreduce(f, seq[1:])
        if last:
            return f(first, last)
        else:
            return first

myreduce(lambda x, y: x*y, list(range(1,6)))

120

## 3.3 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

a) Function composition is a common in both maths and programming. Write a function `compose` which takes two functions, $f$ and $g$, and produces the _composite_ function $f \circ g$, where $(f \circ g)(x) \Leftrightarrow f(g(x))$. Example use is given below.

In [27]:
from statistics import stdev, mean

def compose(f, g):
    return lambda x: f(g(x))

def myscale(vals):
    return [x/stdev(vals) for x in vals]

def myshift(vals):
    return [x-mean(vals) for x in vals]

standardize = compose(myscale, myshift)

print(standardize(range(-3, 8)))

[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


b) Create a function `composition(*funs)` which takes a non-empty sequence of functions of one argument and returns their sequential composition. That is $composition(f_0,f_1, \ldots, f_n) = f_0 \circ f_1 \circ\ldots \circ f_n$. (The question of if $f\circ g \circ h$ should be read $f\circ (g\circ h)$ or $(f \circ g) \circ h$ is perfectly valid, but they turn out to be the same. That is, $\circ$ is associative.)

In [28]:
def compose(f, g):
    return lambda x: g(f(x))

def composition(*funs):
    return reduce(compose , funs, lambda x: x)

Hint: Don't remember what can be found in `*funs`? Print it! Don't know how the values should be combined? Write out some simple example on paper.

Note: This task demonstrates the generality of our constructs. Previously we worked with sequences of numbers and the like. Now we lift this to the level of working with functions as values, and instead of using combinators which work on numbers, we use function combinators in conjunction with our known patterns.

### Voluntary task: pipelining

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

You may want to use functions you have defined above.

In [29]:
from functools import reduce, partial
from math import sqrt 


def make_pipeline(*funs):
    comp = composition(*funs)
    print(comp)
    return lambda vals: map(comp, vals)

# We can even drop the lambda vals : bit, using partial
# evaluation (see the help for functools.partial!)

def inc(x):
    return x+1

pipeline = make_pipeline(inc, abs, sqrt)
rng = list(range(-5, 5))
res = tuple(pipeline(rng))
for i in range(len(rng)):
    print("Value : ", rng[i], " Pipelined to ---> ", res[i])

<function compose.<locals>.<lambda> at 0x000000000510D2F0>
Value :  -5  Pipelined to --->  2.0
Value :  -4  Pipelined to --->  1.7320508075688772
Value :  -3  Pipelined to --->  1.4142135623730951
Value :  -2  Pipelined to --->  1.0
Value :  -1  Pipelined to --->  0.0
Value :  0  Pipelined to --->  1.0
Value :  1  Pipelined to --->  1.4142135623730951
Value :  2  Pipelined to --->  1.7320508075688772
Value :  3  Pipelined to --->  2.0
Value :  4  Pipelined to --->  2.23606797749979


## 4. Simple declarative Pythonic patterns (involing HOF)

a) As preparation, create a named tuple type "coord" which has fields `x` and `y`.

In [16]:
# Add the requisite import statement here
from collections import namedtuple
coord = namedtuple('coord', ['x', 'y']) # your code here

five_three = coord(5,3)
assert five_three.x == 5, "first element is the x coordinate"
assert five_three.y == 3, "the second element is the y coordinate"

b) Generate a $10^7$ random coordinates, with $x$ and $y$ coordinates drawn uniformly from [-1000,1000]. Save the tuple of those with $x + y > 0$ as `rnd_coords`. How many are there?

In [18]:
from random import uniform
def gen_rand_coord(x):
    return coord(uniform(-1000, 1000), uniform(-1000, 1000))

rnd_coords = tuple(filter(lambda crd: (crd.x+crd.y)>0, map(gen_rand_coord, range(10**3))))
len(rnd_coords)

483

[Note: If this takes a while, you might want to consider when the elements are generated and saved.]

**Before having solved the tasks below, consider setting `coords` to a smaller set (eg generate $10^3$ elements instead of $10^7$ to start with).**

c) Let `sorted_rnd` be the coordinated sorted first by the `x` component and then the `y`. Use a built-in Python sorting function. Do you need any extra parameters? Why? Why not? How would you find out where the order comes from (and might it be consistent but useless, eg sorting the elements by memory location)?

In [19]:
coords = list(map(gen_rand_coord, range(10**3)))
sorted_rnd = sorted(coords, key= lambda crd: crd.x)
sorted_rnd = tuple(sorted(sorted_rnd, key= lambda crd: crd.y))
print(sorted_rnd[:10])
print(sorted_rnd[-10:])

(coord(x=-552.6773325690559, y=-998.5599739445969), coord(x=558.3491085139965, y=-997.3606595242964), coord(x=483.81555387701724, y=-996.5332741943806), coord(x=-914.0068422163026, y=-996.476387996907), coord(x=-369.9631637877934, y=-996.2133266054134), coord(x=-104.90986801911765, y=-991.9547264243098), coord(x=-918.3697997010119, y=-988.4750023574298), coord(x=567.9772340031795, y=-980.976879724401), coord(x=-813.8463922690711, y=-979.9432671920985), coord(x=-918.5627649809396, y=-979.9328884220864))
(coord(x=453.67472413826454, y=971.8403591294255), coord(x=905.1297340348322, y=975.573973338282), coord(x=331.5780978611317, y=977.1268904218828), coord(x=-81.78629050982545, y=981.5563083546317), coord(x=-68.62206044154459, y=984.3133521059881), coord(x=728.849338790998, y=986.6020097369867), coord(x=-214.36652783011925, y=986.7567876725891), coord(x=-203.43666450539888, y=986.9209370265457), coord(x=-386.2906241235029, y=988.3180872395301), coord(x=696.1135855769244, y=989.06258308708

[General words of advice:

* During testing, you might want to use a smaller data set (and then try it out at a larger set).
* You might not want to display the entire list to see if you're right all the time. Slicing out the first and last elements, say the first or last 10, might provide some hints.
* You could naturally define a function which checks that the list is in order (or performs some probabilistic sampling test), to test this.]

d) Sort the values (in the sense of returning a new sorted tuple) by their Euclidean distance to the point (5,3). Continue using a built-in Python sorting function.

In [20]:
pts_near_53 = tuple(sorted(coords, key= lambda crd: sqrt((crd.x-5)**2 + (crd.y-3)**2)))
print(pts_near_53[:10])
print(pts_near_53[-10:])

(coord(x=25.530282091706795, y=27.592592144565515), coord(x=31.269315588619975, y=-27.83276107992674), coord(x=28.546092812058532, y=43.47443502776059), coord(x=-45.65213529447874, y=9.636057766470458), coord(x=-10.466790662415406, y=-54.80952384204568), coord(x=14.324766068156691, y=69.35556718708426), coord(x=68.65718849324799, y=-18.796001493476297), coord(x=-74.6757161446792, y=0.6505505232286168), coord(x=-52.216326026552224, y=-70.14801843880684), coord(x=-55.995615483469805, y=-67.79047701650768))
(coord(x=-947.283524006345, y=929.3827871792932), coord(x=-990.1511845365214, y=-888.3358492284843), coord(x=-951.3932372919832, y=945.1486064512387), coord(x=-969.7535750382542, y=928.4415720322224), coord(x=-939.0607573483702, y=-953.8718447008374), coord(x=-930.2021294457328, y=970.5081542111764), coord(x=959.1672021221689, y=-946.5574765917663), coord(x=-918.5627649809396, y=-979.9328884220864), coord(x=-918.3697997010119, y=-988.4750023574298), coord(x=-914.0068422163026, y=-996.4

Note: here we customise the behaviour of a built-in function by passing it information about our intended ordering.

d) Define the function `sorted_by_distance(origo)` which takes a coordinate `origo` and returns a function which sorts the sequence by the euclidean distance to `origo`. (Ie those closest to origo come first in the list.)

In [21]:
def sorted_by_distance(origo):
    return lambda lis: tuple(sorted(lis, key= lambda crd: sqrt((crd.x-origo.x)**2 + (crd.y-origo.y)**2)))

ordered_by_closeness_to_53 = sorted_by_distance(coord(5,3))   # Return the function.
pts_near_53_2 = ordered_by_closeness_to_53(coords)     # Applying the function 

print(pts_near_53_2[:10])
print(pts_near_53_2[-10:])

assert pts_near_53 == pts_near_53_2

(coord(x=25.530282091706795, y=27.592592144565515), coord(x=31.269315588619975, y=-27.83276107992674), coord(x=28.546092812058532, y=43.47443502776059), coord(x=-45.65213529447874, y=9.636057766470458), coord(x=-10.466790662415406, y=-54.80952384204568), coord(x=14.324766068156691, y=69.35556718708426), coord(x=68.65718849324799, y=-18.796001493476297), coord(x=-74.6757161446792, y=0.6505505232286168), coord(x=-52.216326026552224, y=-70.14801843880684), coord(x=-55.995615483469805, y=-67.79047701650768))
(coord(x=-947.283524006345, y=929.3827871792932), coord(x=-990.1511845365214, y=-888.3358492284843), coord(x=-951.3932372919832, y=945.1486064512387), coord(x=-969.7535750382542, y=928.4415720322224), coord(x=-939.0607573483702, y=-953.8718447008374), coord(x=-930.2021294457328, y=970.5081542111764), coord(x=959.1672021221689, y=-946.5574765917663), coord(x=-918.5627649809396, y=-979.9328884220864), coord(x=-918.3697997010119, y=-988.4750023574298), coord(x=-914.0068422163026, y=-996.4

[Note: Here we extend the work above to a higher-order function, which uses the local value of `origo`. In essence, this task summarises higher order functionality - we create a closure, return a function and use a custom ordering ]

e) So far in the course, we have seen, and possibly used `enumerate`, `range`, `zip`, `map` and `filter` as declarative constructs (along with the general comprehension syntax). Now we introduce a further useful iterator construct. Construct something called `reverse_squared` which when prompted would give us the squares of elements 0,...,N _but in reverse_ (that is $N^2, (N-1)^2, ..., 2^2, 1^2, 0^2$).

In [22]:
# The time it takes to run this shouldn't really depend on if you use SMALL_N or BIG_N.

BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    # change this to test later on

reverse_squares = list(map(lambda x: x*x, range(N, -1, -1))) # your code here
print(reverse_squares[-21:])

[400, 361, 324, 289, 256, 225, 196, 169, 144, 121, 100, 81, 64, 49, 36, 25, 16, 9, 4, 1, 0]


In [23]:
# Experimentation: copy and paste your code from above into this cell.
# This is rather crude, but we want you to to be able to trust that any
# slowness in the cell above can be found by reference to that code, not the
# profiling code below.

# Copy-pasting as it might be useful to have fresh maps.


import profile

# We cut and paste this code 
BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    
# Look at the run time. Switching from BIG_N to SMALL_N shouldn't really matter.
# This suggests that we have quick access to elements at the end of our (squared) range.

reverse_squares = list(map(lambda x: x*x, range(N, -1, -1))) # <<----------- Your code from the cell above goes here.


profile.run("print('Did we find it? ', {} in reverse_squares)".format( N**2 ))

Did we find it?  True
         65 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 :0(acquire)
        5    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(exec)
        4    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        1    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        4    0.000    0.000    0.000    0.000 iostream.py:382(write)
        5    0.000    0.000    0.000    0.000 iostream.py

Note: once you know of the construct, this task is extremely simple. It mostly serves as a demonstration of the availability of these constructs, and how they can be combined. Also, it points to efficiency considerations when using declarative iterator constructs as opposed to fixed computed structures.

As the profiling code above suggests, where we redefine the object in every run, we do not have a purely functional construct. In that case, we wouldn't be able to exhaust the values.

[Additional reading: some additional tools are available in the `itertools` module.]

## 5 Mutating function state

A function always has access to the environment in which it was created. Usually, this means that the function can access global variables. It also means that it can access and modify local bindings from where it was created.

A closure is a function which has access to an environment which is not accessible from outside the function (but which is not destroyed when the function returns). I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

a) Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [24]:
def make_counter(n):
    def inc_counter():
        nonlocal n
        n = n+1
        return n
    return inc_counter

counter_A = make_counter(0)
counter_B = make_counter(15)
print("To show that the functions do not affect each others' states, consider the printout:")
print("counter_A returns: {}".format(counter_A()))
print("counter_A returns: {}".format(counter_A()))
print("counter_B returns: {}".format(counter_B()))
print("counter_A returns: {} (was it affected by the call to counter_B?)".format(counter_A()))
print("- No counter_A was not affected by the call to counter_B.")

To show that the functions do not affect each others' states, consider the printout:
counter_A returns: 1
counter_A returns: 2
counter_B returns: 16
counter_A returns: 3 (was it affected by the call to counter_B?)
- No counter_A was not affected by the call to counter_B.


## 6. Use case: parametrisation

Above, we see how `sorted` can be parametrised with information about the intended order. We want to extend our `quicksort` to work the same way. We should be able to provide the way for it to tell if object A in the tuple should come before object B, or after. This is done by mapping the objects onto something where we do have an order.

a) Copy your code from the `quicksort` task above, and extend it. Call the function `quicksort_param` for parametrised, and allow a key parameter to be passed in (like in `sorted`). Note that the key function should be optional. We thus want default arguments.

In [25]:
from random import random

# Write quicksort_param here:
def partition_param(A, piv, f):
    A = list(A)
    temp = A[0]
    A[0] = A[piv]
    A[piv] = temp
    i = 1
    for j in range(1,len(A)):
        if f(A[j]) < f(A[0]):
            if i==j:
                i += 1
            else:
                temp = A[j]
                A[j] = A[i]
                A[i] = temp
                i += 1
    temp = A[0]
    A[0] = A[i-1]
    A[i-1] = temp
    return A, i-1

def quicksort_param(A, f=lambda x: x):
    n = len(A)
    if n<=1:
        return A
    else:
        piv = choice(range(n))
        A, ind = partition_param(A, piv, f)
        A[0:ind] = quicksort_param(A[0:ind], f)
        A[ind+1:] = quicksort_param(A[ind+1:], f)
        return tuple(A)


a = tuple(tuple(random() for i in range(3)) for j in range(10))
print(a)
print("\nSorted using Quicksort Param :\n")
b = quicksort_param(a, sum)   # Elements are three-tuples. Those with smallest sums of values should come first.
for i in b:
    print(i, sum(i))
print(quicksort_param([5,2,9,200]))   # No key function provided.

((0.8464016247568648, 0.6769523093440624, 0.05121930816651432), (0.49666927469300204, 0.565330606723813, 0.9535352224711416), (0.012293412115924163, 0.6092001739792255, 0.8822205976505244), (0.5327750502469324, 0.2200841185304625, 0.5178877957021811), (0.7781970997736402, 0.46834608764334085, 0.8133189617611636), (0.8510649486517419, 0.7664885376820035, 0.4407335670686803), (0.6637531339208227, 0.5844734300521771, 0.16288806818068602), (0.05902649633711643, 0.04833625511214046, 0.36327842337528793), (0.9552427735227573, 0.8915280155779856, 0.5081358537738155), (0.3991700723262681, 0.643469786456928, 0.36843241278785255))

Sorted using Quicksort Param :

(0.05902649633711643, 0.04833625511214046, 0.36327842337528793) 0.4706411748245448
(0.5327750502469324, 0.2200841185304625, 0.5178877957021811) 1.2707469644795761
(0.3991700723262681, 0.643469786456928, 0.36843241278785255) 1.4110722715710486
(0.6637531339208227, 0.5844734300521771, 0.16288806818068602) 1.4111146321536858
(0.01229341211

## Attribution

Lab by Johan Falkenjack (2018), extended and rewritten by Anders Märak Leffler (2019).

License [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)